In [1]:
import ee
# Initialize the Earth Engine module
ee.Initialize()


In [2]:

def remapper(image):
    return image.remap([1, 2, 4, 5, 7, 8, 9, 10, 11], [1, 2, 3, 4, 5, 6, 7, 8, 9])

def export_lulc(city):
    # Filter the administrative boundary for the given city
    aoi = ee.FeatureCollection('projects/earthengine-legacy/assets/projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM2') \
        .filter(ee.Filter.eq('shapeName', city)).geometry()
    # print Area in sq km of aoi
    print(f'Area of {city} in sq km: {aoi.area().divide(1e6).getInfo()}')
    # Load and process the land cover data
    landcover = ee.ImageCollection('projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS') \
        .filterDate('2022-01-01', '2022-12-31') \
        .map(remapper) \
        .mosaic() \
        .clip(aoi) \
        .rename('b1')

    # Define the export task
    task = ee.batch.Export.image.toDrive(
        image=landcover,
        description=f'{city}_LULC',
        folder='FMSE_LULC',
        fileNamePrefix=f'{city}_LULC',
        region=aoi,
        scale=10,
        maxPixels=1e13
    )

    # Start the export task
    task.start()
    print(f'Export task started for {city}.')

# List of cities
cities = ['Shikarpur', 'Sylhet', 'Ernakulam', 'Nhamatanda']

# Iterate through the cities and export LULC data for each
for city in cities:
    export_lulc(city)


Area of Shikarpur in sq km: 2576.278345966003
Export task started for Shikarpur.
Area of Sylhet in sq km: 3440.8448022050648
Export task started for Sylhet.
Area of Ernakulam in sq km: 2388.041910361694
Export task started for Ernakulam.
Area of Nhamatanda in sq km: 4026.219014010595
Export task started for Nhamatanda.
